Already exists:
- Boligrafica Twitter Tweepy
- Greenhouse Summon
- NOAA Data update GHD

Want to create:
- GFW - Datos Bolivia Deforestacion / Incendios
- Other twitter bots!??

Types of conditions:
- Runs on a schedule 
  - once every week (like boligrafica)
  - 4 times a day (like greenhouse summon)
- Runs if it has detected updated data (if new data, run)
  - Like NOAA data in GHD

In [4]:
import os
import sys
import pandas as pd
import datetime
from IPython import get_ipython

# environmental secrets when working locally
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
def NEW_DATA_RUNNER():
    
    return

In [6]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting: ", notebook_path)
    
    if schedule_in_days == None:
        EXECUTE = True
        
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = ["2002-07-16 18:32",np.nan]
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        import datetime
        now = datetime.datetime.now()
        last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # FINAL CHECK
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            

        
    print("Execute is: ", str(EXECUTE))
    if EXECUTE == True:

        
        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "SUCCESS!"
            print(status)
        except:
            status = "FAILURE."
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")
    else:
        print("Did not execute nor log\n\n")
    return

########################################################
#

SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Twitter/GreenhouseSummon.ipynb", schedule_in_days= .0006/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb", schedule_in_days= 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb", schedule_in_days= 1)
SCHEDULED_RUNNER(notebook_path = "BoliGrafica/Twitter/twitter_post.ipynb", schedule_in_days= .1/24)


starting:  Greenhouse_Data/Twitter/GreenhouseSummon.ipynb
previous log found: 2022-08-03 18:21
difference in execution is: 0.0031 days
needed difference to execute: 0.0 days
Execute is:  True
envs failed to load
Iran, Islamic Republic of greenhouse gas emission data and figures 👇

If interested be sure to check out my website for many more datasets and figures! Thank you and take care! 🤖
Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Panama"


/home/daqub/anaconda3/lib/python3.9/site-packages/tweepy/auth.py:120: DeprecationWarning: OAuthHandler is deprecated; use OAuth1UserHandler instead.
  warnings.warn(


 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Seychelles"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Virgin Islands, British"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Taiwan"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions French Southern Territories"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Sudan"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Vincent and the Grenadines"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list 

['country_data/EGY_Egypt/figures/EGY_relative_totals.png',
 'country_data/EGY_Egypt/figures/EGY_GCP_Country_Highlight.png',
 'country_data/EGY_Egypt/figures/EGY_GCP_1.png',
 'country_data/EGY_Egypt/figures/EGY_Minx_top20_subsectors.png']

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [False] 



Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Burundi"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Thailand"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Aruba"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Andorra"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Iceland"
 > found 1 tweets
 > days of invalidity: 20
 > Number of invalids: 112
 > 0 usernames filtered out
 > retweets removed: 0


...

['country_data/DEU_Germany/figures/DEU_relative_totals.png',
 'country_data/DEU_Germany/figures/DEU_GCP_Country_Highlight.png',
 'country_data/DEU_Germany/figures/DEU_GCP_1.png',
 'country_data/DEU_Germany/figures/DEU_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [False, True, True] 





 FINISHED WHILE LOOP


 FINISHED WHILE LOOP
TOOO SOON!!
SUCCESS!
New log entry successful!


starting:  Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb
previous log found: 2022-08-03 08:15
difference in execution is: 0.4243 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


starting:  Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb
previous log found: 2022-08-03 18:07
difference in execution is: 0.0132 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


starting:  BoliGrafica/Twitter/twitter_post.ipynb
previous log found: 2022-08-03 18:18
difference in execution is: 0.0056 days
needed difference to execute: 0.0042 days
Execute is:  True
envs failed to load
{"created_at": "Fri Jul 10 05:56:09 +0000 2020", "default_profile": true, "description": "Monitoreo COVID-19 en Bolivia. Datos abiertos. \u00a1Visite la p\u00e1gina

,Bolivia,La Paz,Cochabamba,Santa Cruz,Oruro,Potosí,Chuquisaca,Tarija,Beni,Pando
2021-02-02,1589,12,278,647,26,138,122,90,6,270
2021-02-03,3560,361,687,1264,106,203,303,200,48,388
2021-02-04,5172,1030,1044,1393,195,293,304,362,63,488
2021-02-05,7430,1783,1369,1638,300,396,529,711,154,550
2021-02-06,7636,1859,1369,1638,412,396,530,712,170,550
...,...,...,...,...,...,...,...,...,...,...
2022-07-24,6276782,1524531,1208274,1802763,327107,379189,362686,365435,229925,76872
2022-07-25,6278211,1524981,1208603,1803027,327175,379268,362781,365509,229970,76897
2022-07-26,6280746,1525716,1209060,1803721,327298,379376,362911,365663,230052,76949
2022-07-27,6283591,1526532,1209531,1804656,327414,379502,363057,365748,230136,77015


,Bolivia,La Paz,Cochabamba,Santa Cruz,Oruro,Potosí,Chuquisaca,Tarija,Beni,Pando
2020-03-10,0,0,0,0,0,0,0,0,0,0
2020-03-11,1,0,0,1,0,0,0,0,0,0
2020-03-12,0,0,0,0,0,0,0,0,0,0
2020-03-13,7,0,1,0,6,0,0,0,0,0
2020-03-14,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2022-07-28,6014,883,2081,1428,277,224,447,524,77,73
2022-07-29,5331,896,1789,1153,248,280,385,463,67,50
2022-07-30,3570,1018,910,587,253,229,270,174,125,4
2022-07-31,1907,469,665,384,143,105,104,33,1,3


last update vacunacion:  2022-07-28
last update covid epi:  2022-08-01
today is Wednesday in native timezone (which is two hours before zuritime)
SUCCESS!
New log entry successful!


